![EDA](../5_Sources/Images/banner_eda.gif)

<p align="center">

## **Normalización de Datos - EDA**

</p>

Se evidencia que en varias columnas su contenido está estructurado como diccionario y como formato JSON, la prioridad es analizar estas columnas si son datos relevantes y si es posible desanidarlas en el mismo dataset sin repetir datos se realiza.

<mark>Parte de este script será reutilizado para agregar a la automatización</mark>

**`Importante:`** Para que este **EDA** se pueda realizar es necesario tener en cuenta la data analizada de `EDA_hotelbeads_details.ipynb` a su vez el diccionario de datos realizado en [Excel aquí](https://docs.google.com/spreadsheets/d/1Cb9qUBbyXu-8US3OhfAiyZLHjSJrZstZ/edit?usp=sharing&ouid=105689737903283188966&rtpof=true&sd=true).

In [26]:
# Usaremos librería Pandas y Json para permitir la lectura de los archivos
import pandas as pd
import json
import re

In [27]:
# Ruta base donde se encuentran los archivos JSON
path_base = "../2_Datasets/original/hotelbeds/"

### Realizamos lectura del dataset

In [ ]:
# Guardamos el dataset en csv para la fácil lectura.
df_hotels_c = pd.read_csv(path_base + "hotels_dataset.csv")

### Al Menjar Grandes cantidades de Datos el PC local comienza a limitarse en memoria RAM y en Procesador por lo cual realizamos la toma de una muestra de uno de los lotes de datasets (1 archivo JSON) y procesamos el script pertinente y luego lo corremos en la concatenación de la data, esto reduce los recursos.

In [29]:
df_hotels_c = pd.read_json(path_base + "hotels_us_total_api_1.json")

In [30]:
# Revisamos su estructura, tipos de datos
df_hotels_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   code                   1000 non-null   int64 
 1   name                   1000 non-null   object
 2   description            1000 non-null   object
 3   countryCode            1000 non-null   object
 4   stateCode              1000 non-null   object
 5   destinationCode        1000 non-null   object
 6   zoneCode               1000 non-null   int64 
 7   coordinates            1000 non-null   object
 8   categoryCode           1000 non-null   object
 9   categoryGroupCode      999 non-null    object
 10  chainCode              797 non-null    object
 11  accommodationTypeCode  1000 non-null   object
 12  boardCodes             977 non-null    object
 13  segmentCodes           1000 non-null   object
 14  address                1000 non-null   object
 15  postalCode            

Analizamos columnas como `license` revisamos el diccionario de datos y es una licencia interna de Hotelbeds, vemos que los campos vacíos están en más del 99% por lo cual se considera eliminar la columna, tampoco es relevante para los KPIs y los objetivos.

In [31]:
df_hotels_c.head()

,code,name,description,countryCode,stateCode,destinationCode,zoneCode,coordinates,categoryCode,categoryGroupCode,...,interestPoints,images,wildcards,web,lastUpdate,S2C,ranking,terminals,issues,license
0,6474,{'content': 'Hilton Chicago'},{'content': 'Our Hotel's Policies Have Changed...,US,IL,ORD,2,"{'longitude': -87.6244, 'latitude': 41.8725}",4EST,GRUPO4,...,"[{'facilityCode': 10, 'facilityGroupCode': 100...","[{'imageTypeCode': 'HAB', 'path': '00/006474/0...","[{'roomType': 'ROO.LK', 'roomCode': 'ROO', 'ch...",www3.hilton.com/en/hotels/illinois/hilton-chic...,2023-07-28,3*,47,NaN,NaN,NaN
1,6478,{'content': 'Four Points by Sheraton Los Angel...,{'content': 'This charming hotel is just a mil...,US,CA,LAX,9,"{'longitude': -118.3859708, 'latitude': 33.948...",3EST,GRUPO3,...,NaN,"[{'imageTypeCode': 'RES', 'path': '00/006478/0...",NaN,http://www.fourpointslax.com/la-airport-hotel,2023-07-28,3*,82,NaN,NaN,NaN
2,6480,{'content': 'Sheraton Universal Hotel'},"{'content': 'This upscale, landmark hotel enjo...",US,CA,LAX,21,"{'longitude': -118.35955291287974, 'latitude':...",3EST,GRUPO3,...,NaN,"[{'imageTypeCode': 'GEN', 'path': '00/006480/0...",NaN,NaN,2023-07-28,4*,119,NaN,NaN,NaN
3,6483,{'content': 'Westin Copley Place'},{'content': 'Experience one of Boston's most c...,US,MA,BOS,2,"{'longitude': -71.077539, 'latitude': 42.348577}",H4_5,GRUPO4,...,NaN,"[{'imageTypeCode': 'GEN', 'path': '00/006483/0...","[{'roomType': 'DBL.ST', 'roomCode': 'DBL', 'ch...",http://www.westincopleyplaceboston.com/,2023-07-28,3*,62,NaN,NaN,NaN
4,6487,{'content': 'Wyndham Garden New Orleans Airport'},{'content': 'Wyndham Garden New Orleans Airpor...,US,LA,MSY,8,"{'longitude': -90.1227, 'latitude': 30}",3EST,GRUPO3,...,NaN,"[{'imageTypeCode': 'RES', 'path': '00/006487/0...","[{'roomType': 'DBL.KG', 'roomCode': 'DBL', 'ch...",www.wyndhamhotels.com,2023-07-28,NaN,55,NaN,NaN,NaN


Preparamos un script para poder determinar que columnas están anidadas en forma de diccionarios, listas o formatos JSON

In [41]:
# Define una función para determinar el tipo de contenido anidado
def determine_nested_type(cell):
    if isinstance(cell, list) and all(isinstance(item, dict) for item in cell):
        return 'Lista de diccionarios'
    elif isinstance(cell, dict):
        return 'Diccionario'
    elif isinstance(cell, list):
        return 'Lista'
    
    return 'Desconocido'

# Crea un diccionario para almacenar los resultados por columna
results = {}

# Itera a través de las columnas del DataFrame original
for column in df_hotels_c.columns:
    # Inicializa un conjunto para esta columna (conjunto para eliminar duplicados)
    nesteds_type = set()
    
    # Itera a través de las celdas de la columna
    for cell in df_hotels_c[column]:
        # Determina el tipo de contenido anidado
        nested_type = determine_nested_type(cell)
        
        # Si el tipo no es "Desconocido", agrega al conjunto
        if nested_type != 'Desconocido':
            nesteds_type.add(nested_type)
    
    # Almacena el resultado para esta columna en el diccionario de resultados
    results[column] = ', '.join(nesteds_type) if nesteds_type else 'Ninguno'

# Crea un DataFrame a partir del diccionario de resultados
resume_df = pd.DataFrame(list(results.items()), columns=['Columnas', 'Tipos de Contenido Anidado'])

# Muestra el DataFrame resumen
print(resume_df)

                 Columnas    Tipos de Contenido Anidado
0                    code                       Ninguno
1                    name                   Diccionario
2             description                   Diccionario
3             countryCode                       Ninguno
4               stateCode                       Ninguno
5         destinationCode                       Ninguno
6                zoneCode                       Ninguno
7             coordinates                   Diccionario
8            categoryCode                       Ninguno
9       categoryGroupCode                       Ninguno
10              chainCode                       Ninguno
11  accommodationTypeCode                       Ninguno
12             boardCodes                         Lista
13           segmentCodes  Lista, Lista de diccionarios
14                address                   Diccionario
15             postalCode                       Ninguno
16                   city                   Dicc

La **lista** unicamente son referencia de código que hace un llamado a otra tabla relacionada es el caso de `boardCodes` y `segmentCodes`

La **Lista de Diccionarios** hace un llamado a más parametros o descripciones de servicios que tiene el Hotel, estos se pueden manejar en su mayoría como una tabla relacionada, este es un proceso de ETL y normalizado de tablas.

Podemos observar que las columnas que son en forma de **Diccionarios** son: `name`, `description`, `coordinates`, `address`, `city` estas columnas se procede a desanidar para que quede en formato de string

In [42]:
# Define una función para extraer el contenido de 'content'
def extract_content(json_dict):
    if isinstance(json_dict, dict):
        return json_dict.get('content', '')
    else:
        return ''

In [43]:
# Aplica la función a la columna y crea una nueva columna con el contenido extraído
df_hotels_c['name'] = df_hotels_c['name'].apply(extract_content)
df_hotels_c['description'] = df_hotels_c['description'].apply(extract_content)
df_hotels_c['address'] = df_hotels_c['address'].apply(extract_content)
df_hotels_c['city'] = df_hotels_c['city'].apply(extract_content)

In [44]:
# Separamos coordinates en dos columnas respectivamente longitude y latitude
def extract_longitude(json_dict):
    if isinstance(json_dict, dict):
        return json_dict.get('longitude', '')
    else:
        return ''
    
def extract_latitude(json_dict):
    if isinstance(json_dict, dict):
        return json_dict.get('latitude', '')
    else:
        return ''

In [45]:
# Aplicamos la Función
df_hotels_c['longitude'] = df_hotels_c['coordinates'].apply(extract_longitude)
df_hotels_c['latitude'] = df_hotels_c['coordinates'].apply(extract_latitude)

Eliminamos algunas columnas que ya se encuentran en el dataset de `hotels_details_dataset.csv`, es el caso de `terminals, issues, wildcards, images, interestPoints, facilities, rooms` en otros casos como el de `license` por tener el 99% de campos vacíos se procede a eliminar la columna ya verificando que los datos no son relevantes, `coordinates` se elimina ya que se dividión en dos columnas.

In [18]:
# Eliminar columnas con datos null: 'terminals', 'license', 'issues', 'wildcards', 'images', 'interestPoints', 'facilities', 'rooms'
df_hotels_c = df_hotels_c.drop(columns=['terminals', 'license', 'issues', 'wildcards', 'images', 'interestPoints', 'facilities', 'rooms'])
# Eliminar Columna ya gestionada y dividida
df_hotels_c = df_hotels_c.drop(columns=['coordinates'])

In [19]:
df_hotels_c.head()

,code,name,description,countryCode,stateCode,destinationCode,zoneCode,categoryCode,categoryGroupCode,chainCode,...,postalCode,city,email,phones,web,lastUpdate,S2C,ranking,longitude,latitude
0,6474,Hilton Chicago,Our Hotel's Policies Have Changed ” Weve upda...,US,IL,ORD,2,4EST,GRUPO4,HILTO,...,60605,CHICAGO,CHICH_GM@hilton.com,"[{'phoneNumber': '+13129224400', 'phoneType': ...",www3.hilton.com/en/hotels/illinois/hilton-chic...,2023-07-28,3*,47,-87.624400,41.872500
1,6478,Four Points by Sheraton Los Angeles Int'l Airport,This charming hotel is just a mile from LAX Ai...,US,CA,LAX,9,3EST,GRUPO3,MARIO,...,90045,LOS ANGELES,NaN,"[{'phoneNumber': '+0017147038800', 'phoneType'...",http://www.fourpointslax.com/la-airport-hotel,2023-07-28,3*,82,-118.385971,33.948050
2,6480,Sheraton Universal Hotel,"This upscale, landmark hotel enjoys an excitin...",US,CA,LAX,21,3EST,GRUPO3,MARIO,...,91608,UNIVERSAL CITY,NaN,"[{'phoneNumber': '+7145227000', 'phoneType': '...",NaN,2023-07-28,4*,119,-118.359553,34.137603
3,6483,Westin Copley Place,Experience one of Boston's most celebrated nei...,US,MA,BOS,2,H4_5,GRUPO4,MARIO,...,2116,BOSTON,NaN,"[{'phoneNumber': '+16172879100', 'phoneType': ...",http://www.westincopleyplaceboston.com/,2023-07-28,3*,62,-71.077539,42.348577
4,6487,Wyndham Garden New Orleans Airport,Wyndham Garden New Orleans Airport is perfectl...,US,LA,MSY,8,3EST,GRUPO3,WYNDH,...,70003,METAIRIE,NaN,"[{'phoneNumber': '+15045240427', 'phoneType': ...",www.wyndhamhotels.com,2023-07-28,NaN,55,-90.122700,30.000000
